In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')


import gensim
from gensim.models import Word2Vec

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('../Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','class'] )
df_test = pd.read_csv('../Data/pcl_test.tsv', sep = '\t',names=['id','info','country', 'text'] )

In [3]:
df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10468 entries, 0 to 10467
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10468 non-null  object
 1   info     10468 non-null  object
 2   country  10468 non-null  object
 3   text     10468 non-null  object
 4   class    10468 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 409.0+ KB


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Preetam
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

corpus =[]
from tqdm import tqdm 
for i in tqdm(range(0,10468)):
    
    review = re.sub('[^a-zA-Z]',' ', df['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set (all_stopwords)]
    review =' '.join(review)
    corpus.append(review)
    

100%|███████████████████████████████████████████████████████████████████████████| 10468/10468 [00:18<00:00, 553.30it/s]


In [6]:
cv = CountVectorizer( max_features = 1000)
#cv = TfidfVectorizer(min_df=1,stop_words='english')
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:,-1].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 0)
   

y_train_task1 = [ 0 if (y == 1 or y == 0) else 1 for y in y_train ]
y_test_task1 = [ 0 if (y == 1 or y == 0) else 1 for y in y_test ]


In [8]:
logmodel = LogisticRegression(max_iter=1000, class_weight='balanced')
logmodel.fit(X_train, y_train_task1)
predictions = logmodel.predict(X_test)

In [9]:
accuracy = confusion_matrix(y_test_task1, predictions)
print(accuracy)

[[1560  330]
 [  95  109]]


In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_task1,predictions)
accuracy


0.7970391595033429

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test_task1, predictions))


              precision    recall  f1-score   support

           0       0.94      0.83      0.88      1890
           1       0.25      0.53      0.34       204

    accuracy                           0.80      2094
   macro avg       0.60      0.68      0.61      2094
weighted avg       0.87      0.80      0.83      2094



In [14]:
sum(df_test['text'].isnull())

0

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3832 entries, t_0 to t_3897
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       3832 non-null   object
 1   info     3832 non-null   object
 2   country  3832 non-null   object
 3   text     3832 non-null   object
dtypes: object(4)
memory usage: 278.7+ KB


In [14]:
df_test= df_test.dropna()

In [15]:
df_test.isnull()

,id,info,country,text
t_0,False,False,False,False
t_1,False,False,False,False
t_2,False,False,False,False
t_3,False,False,False,False
t_4,False,False,False,False
...,...,...,...,...
t_3893,False,False,False,False
t_3894,False,False,False,False
t_3895,False,False,False,False
t_3896,False,False,False,False


In [15]:
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

corpus_test =[]
from tqdm import tqdm 
for i in tqdm(range(0,3832)):
    
    review = re.sub('[^a-zA-Z]',' ', df['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set (all_stopwords)]
    review =' '.join(review)
    corpus_test.append(review)
    

100%|█████████████████████████████████████████████████████████████████████████████| 3832/3832 [00:05<00:00, 648.97it/s]


In [16]:
#cv = CountVectorizer( max_features = 1000)
#cv = TfidfVectorizer(min_df=1,stop_words='english')
x_test = cv.fit_transform(corpus_test).toarray()
#y = df.iloc[:,-1].values

In [17]:
#x_test = df_test[['text']]

In [17]:
y_pred = logmodel.predict(x_test)

In [18]:
print(y_pred)

[0 0 0 ... 0 0 0]


In [53]:
len(y_pred)

3832

In [20]:
d  = {}
for i in y_pred:
    if i in d:
        d[i] += 1
    else:
        d[i] = 1
    

In [21]:
d

{0: 3437, 1: 395}

In [22]:
S= open("logistic_cv.txt", "w")
for i in y_pred:
    print(i,file= S)
S.close()